# 1. Setting the Working Environment

In [63]:
!pip install pyLDAvis
!pip install wordcloud
!pip install gensim
!pip install nltk


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [64]:

!pip install wordcloud


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [65]:
import pyLDAvis
import pyLDAvis.gensim_models 
import pickle
import pandas as pd
import os
import re
from wordcloud import WordCloud
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import gensim.corpora as corpora
import warnings
warnings.filterwarnings('ignore')

# 2. Importing Data

In [66]:
# Master Folder PathC
path = "E:\\NCKH\\NCKHNew\\labeled_data_final.csv"
df = pd.read_csv(os.path.join(path))
print("The shape of data --->",df.shape)
df.head(10) # Print head

The shape of data ---> (470942, 7)


,index,reviewer_name,reviewer_country,rental_date,comments,language,sentiment
0,0,Igor,Kraków- Ba Lan,02/12/2023 00:00,best support ever witnessed answer fast kind s...,en,positive
1,1,Kevin,Cannes- Pháp,04/12/2023 00:00,everything ok,en,positive
2,2,Walid,Cannes- Pháp,09/12/2023 00:00,great accommodation games owner responded quickly,en,positive
3,3,Maxence,Vương quốc Anh,11/12/2023 00:00,thanks nigel tra friendly stay ho chi minh cit...,en,positive
4,4,Lisa,Ai/len,10/12/2023 00:00,nigel others dedicated responded quickly stay ...,en,positive
5,5,Linh,Thành phố Hồ Chí Minh- Việt Nam,20/12/2023 00:00,ok room price,en,positive
6,6,Heonsoo,Thành phố Hồ Chí Minh- Việt Nam,24/12/2023 00:00,good,en,positive
7,7,Đức,Hà Nội- Việt Nam,18/11/2023 00:00,best option budget close range everything real...,en,positive
8,8,Mark,Thành phố Hồ Chí Minh- Việt Nam,30/11/2023 00:00,great time homeowners quickly everything solve...,en,positive
9,9,Delon,Singapore,19/11/2023 00:00,good accommodation,en,positive


# 3. Inital Pre-Processing Data

In [67]:
#Remove the columns
df = df.drop(['index','reviewer_name','reviewer_country', 'language'], axis = 1)
df
##### Remove punctuation/lower casing
df['df_text_processed'] = \
df['comments'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
df['df_text_processed'] = \
df['df_text_processed'].map(lambda x: x.lower())

# Print out the first rows of df
df['df_text_processed'].head()

0    best support ever witnessed answer fast kind s...
1                                       everything ok 
2    great accommodation games owner responded quickly
3    thanks nigel tra friendly stay ho chi minh cit...
4    nigel others dedicated responded quickly stay ...
Name: df_text_processed, dtype: object

# 4. Initating the LDA analysis

In [68]:
stop_words = stopwords.words('english')
"""
    Functions for converting sentence to words
"""  
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
"""
    Functions removing the stopwords 
"""        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = df.df_text_processed.values.tolist()#creating the array of token 
data_words = list(sent_to_words(data))#converting them into list

data_words = remove_stopwords(data_words)# remove stop words
print(data_words[:1][0][:30])

['best', 'support', 'ever', 'witnessed', 'answer', 'fast', 'kind', 'support', 'accommodation', 'close', 'city', 'center', 'minutes', 'walk', 'although', 'place', 'quiet', 'perfect', 'place', 'rest', 'sightseeing', 'party', 'air', 'conditioning', 'projector', 'works', 'well', 'owner', 'provides', 'netflix']


In [69]:
# Create Dictionary 
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Printing the corpus in a numeric form which implies the frequency of the word
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1)]


In [70]:
#Showing the code
id2word

# 5. LDA model training

In [71]:
from pprint import pprint
# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.103*"friendly" + 0.079*"staff" + 0.065*"helpful" + 0.064*"host" + '
  '0.039*"clean" + 0.020*"room" + 0.019*"incredible" + 0.017*"fridge" + '
  '0.017*"nice" + 0.015*"hotel"'),
 (1,
  '0.124*"nice" + 0.085*"pool" + 0.046*"apartment" + 0.044*"gym" + '
  '0.043*"view" + 0.027*"clean" + 0.026*"place" + 0.024*"swimming" + '
  '0.022*"awesome" + 0.021*"building"'),
 (2,
  '0.183*"good" + 0.098*"great" + 0.084*"location" + 0.073*"place" + '
  '0.037*"stay" + 0.030*"everything" + 0.027*"host" + 0.024*"value" + '
  '0.020*"nice" + 0.020*"price"'),
 (3,
  '0.035*"check" + 0.031*"back" + 0.029*"stay" + 0.027*"time" + 0.025*"come" + '
  '0.024*"clean" + 0.021*"comfortable" + 0.020*"room" + 0.018*"definitely" + '
  '0.017*"mall"'),
 (4,
  '0.083*"clean" + 0.065*"room" + 0.046*"comfortable" + 0.025*"convenient" + '
  '0.020*"spacious" + 0.019*"location" + 0.017*"apartment" + 0.017*"tidy" + '
  '0.015*"big" + 0.015*"terminal"'),
 (5,
  '0.062*"recommend" + 0.050*"highly" + 0.026*"place" + 

# 6. Analysing the LDA model

In [77]:
import os
import pickle
import pyLDAvis
import pyLDAvis.gensim_models

# Khai báo đường dẫn đến thư mục lưu trữ dữ liệu
LDAvis_data_dir = os.path.join("E:\\NCKH\\NCKHNew\\img", '02_Results')
if not os.path.exists(LDAvis_data_dir):
    os.makedirs(LDAvis_data_dir)

# Đường dẫn tới tệp dữ liệu của pyLDAvis
LDAvis_data_filepath = os.path.join(LDAvis_data_dir, 'ldavis_prepared_'+str(num_topics))

# Kiểm tra xem liệu quá trình chuẩn bị dữ liệu cho trực quan hóa đã được thực hiện chưa
if 1 == 1:
    # Chuẩn bị dữ liệu cho trực quan hóa
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    # Lưu trữ dữ liệu vào một tệp
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# Load dữ liệu đã chuẩn bị từ tệp
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

# Lưu trữ trực quan hóa dưới dạng một tệp HTML
pyLDAvis.save_html(LDAvis_prepared, 'ldavis_prepared_'+ str(num_topics) +'.html')

# Hiển thị trực quan hóa trên trình duyệt web
pyLDAvis.display(LDAvis_prepared)


# 8. Topic modelling positive comments

## Data

In [86]:
df_pos = df.loc[df['sentiment'] == 'positive']
df_pos

,rental_date,comments,sentiment,df_text_processed
0,02/12/2023 00:00,best support ever witnessed answer fast kind s...,positive,best support ever witnessed answer fast kind s...
1,04/12/2023 00:00,everything ok,positive,everything ok
2,09/12/2023 00:00,great accommodation games owner responded quickly,positive,great accommodation games owner responded quickly
3,11/12/2023 00:00,thanks nigel tra friendly stay ho chi minh cit...,positive,thanks nigel tra friendly stay ho chi minh cit...
4,10/12/2023 00:00,nigel others dedicated responded quickly stay ...,positive,nigel others dedicated responded quickly stay ...
...,...,...,...,...
470937,25/12/2023 00:00,easy travel around convenient check nice coffe...,positive,easy travel around convenient check nice coffe...
470938,21/12/2023 00:00,easy find white building cars park side road f...,positive,easy find white building cars park side road f...
470939,25/12/2023 00:00,delighted visit welcoming quaint place time vi...,positive,delighted visit welcoming quaint place time vi...
470940,24/12/2023 00:00,nice place city center rooftop pool fitness go...,positive,nice place city center rooftop pool fitness go...


In [88]:
stop_words = stopwords.words('english')
"""
    Functions for converting sentence to words
"""  
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
"""
    Functions removing the stopwords 
"""        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = df_pos.df_text_processed.values.tolist()#creating the array of token 
data_words = list(sent_to_words(data))#converting them into list

data_words = remove_stopwords(data_words)# remove stop words
print(data_words[:1][0][:30])

['best', 'support', 'ever', 'witnessed', 'answer', 'fast', 'kind', 'support', 'accommodation', 'close', 'city', 'center', 'minutes', 'walk', 'although', 'place', 'quiet', 'perfect', 'place', 'rest', 'sightseeing', 'party', 'air', 'conditioning', 'projector', 'works', 'well', 'owner', 'provides', 'netflix']


In [89]:
# Create Dictionary 
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Printing the corpus in a numeric form which implies the frequency of the word
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1)]


In [ ]:
#Showing the code
id2word

## LDA training

In [90]:
from pprint import pprint
# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.025*"hcm" + 0.014*"apartment" + 0.012*"also" + 0.012*"place" + '
  '0.012*"stay" + 0.011*"get" + 0.009*"area" + 0.009*"great" + 0.007*"easy" + '
  '0.007*"food"'),
 (1,
  '0.031*"et" + 0.028*"friendly" + 0.027*"super" + 0.027*"est" + 0.025*"la" + '
  '0.023*"tr" + 0.022*"owner" + 0.021*"le" + 0.016*"hosts" + 0.015*"staff"'),
 (2,
  '0.047*"convenient" + 0.043*"near" + 0.042*"location" + 0.035*"restaurants" '
  '+ 0.026*"clean" + 0.024*"shopping" + 0.024*"close" + 0.024*"many" + '
  '0.022*"mall" + 0.020*"nearby"'),
 (3,
  '0.046*"area" + 0.037*"tourist" + 0.036*"place" + 0.027*"stay" + '
  '0.022*"clean" + 0.021*"station" + 0.019*"walk" + 0.015*"apartment" + '
  '0.015*"location" + 0.015*"recommend"'),
 (4,
  '0.273*"good" + 0.056*"location" + 0.036*"place" + 0.024*"everything" + '
  '0.023*"room" + 0.022*"clean" + 0.018*"nice" + 0.017*"comfortable" + '
  '0.017*"plenty" + 0.015*"service"'),
 (5,
  '0.048*"check" + 0.037*"room" + 0.028*"clean" + 0.018*"like" + 0.015*"host" '


In [91]:
import os
import pickle
import pyLDAvis
import pyLDAvis.gensim_models

# Khai báo đường dẫn đến thư mục lưu trữ dữ liệu
LDAvis_data_dir = os.path.join("E:\\NCKH\\NCKHNew\\img", 'pos_Results')
if not os.path.exists(LDAvis_data_dir):
    os.makedirs(LDAvis_data_dir)

# Đường dẫn tới tệp dữ liệu của pyLDAvis
LDAvis_data_filepath = os.path.join(LDAvis_data_dir, 'ldavis_prepared_'+str(num_topics))

# Kiểm tra xem liệu quá trình chuẩn bị dữ liệu cho trực quan hóa đã được thực hiện chưa
if 1 == 1:
    # Chuẩn bị dữ liệu cho trực quan hóa
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    # Lưu trữ dữ liệu vào một tệp
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# Load dữ liệu đã chuẩn bị từ tệp
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

# Lưu trữ trực quan hóa dưới dạng một tệp HTML
pyLDAvis.save_html(LDAvis_prepared, 'ldavis_prepared_'+ str(num_topics) +'.html')

# Hiển thị trực quan hóa trên trình duyệt web
pyLDAvis.display(LDAvis_prepared)


# 9. Topic modelling negative comments

In [103]:
df_neg = df.loc[df['sentiment'] == 'negative']
df_neg

,rental_date,comments,sentiment,df_text_processed
14,24/12/2023 00:00,small accommodation ventilation system bad sme...,negative,small accommodation ventilation system bad sme...
40,21/12/2023 00:00,problems door air conditioning key temperature...,negative,problems door air conditioning key temperature...
135,09/12/2023 00:00,projector bit blurry expected checking problem...,negative,projector bit blurry expected checking problem...
172,21/12/2023 00:00,good location difficult enter door quite diffi...,negative,good location difficult enter door quite diffi...
349,25/11/2023 00:00,bathroom best stayed well going good bathroom ...,negative,bathroom best stayed well going good bathroom ...
...,...,...,...,...
470816,24/11/2023 00:00,misleading everything thing falling apart woul...,negative,misleading everything thing falling apart woul...
470839,27/11/2023 00:00,die unterkunft befindet sich sehr nah einer me...,negative,die unterkunft befindet sich sehr nah einer me...
470851,26/11/2023 00:00,stay okay get pay room bit loud right next bus...,negative,stay okay get pay room bit loud right next bus...
470914,20/12/2023 00:00,le jour tait parfait merci pour tout je revien...,negative,le jour tait parfait merci pour tout je revien...


In [104]:
stop_words = stopwords.words('english')
"""
    Functions for converting sentence to words
"""  
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
"""
    Functions removing the stopwords 
"""        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = df_neg.df_text_processed.values.tolist()#creating the array of token 
data_words = list(sent_to_words(data))#converting them into list

data_words = remove_stopwords(data_words)# remove stop words
print(data_words[:1][0][:30])

['small', 'accommodation', 'ventilation', 'system', 'bad', 'smell', 'like', 'lizard', 'friendly', 'landlords', 'changed', 'us', 'still', 'bad', 'experience', 'wasting', 'time']


In [105]:
# Create Dictionary 
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Printing the corpus in a numeric form which implies the frequency of the word
print(corpus[:1][0][:30])

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]


In [106]:
#Showing the code
id2word

In [107]:
from pprint import pprint
# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.016*"room" + 0.012*"und" + 0.008*"die" + 0.007*"der" + 0.007*"wir" + '
  '0.007*"host" + 0.006*"das" + 0.006*"ist" + 0.006*"apartment" + 0.005*"es"'),
 (1,
  '0.020*"und" + 0.016*"die" + 0.016*"sehr" + 0.009*"war" + 0.009*"der" + '
  '0.009*"ist" + 0.008*"de" + 0.008*"et" + 0.008*"es" + 0.008*"wir"'),
 (2,
  '0.024*"und" + 0.019*"ist" + 0.013*"die" + 0.013*"der" + 0.009*"das" + '
  '0.008*"sehr" + 0.007*"es" + 0.007*"room" + 0.006*"nicht" + 0.005*"wir"'),
 (3,
  '0.020*"die" + 0.015*"und" + 0.012*"ist" + 0.011*"sehr" + 0.009*"der" + '
  '0.009*"war" + 0.007*"zu" + 0.006*"unterkunft" + 0.006*"wir" + 0.005*"hcm"'),
 (4,
  '0.028*"de" + 0.015*"est" + 0.015*"la" + 0.015*"et" + 0.011*"le" + '
  '0.010*"tr" + 0.010*"un" + 0.009*"tout" + 0.008*"en" + 0.008*"place"'),
 (5,
  '0.011*"place" + 0.009*"host" + 0.006*"die" + 0.006*"room" + 0.006*"ist" + '
  '0.005*"airbnb" + 0.005*"sehr" + 0.005*"und" + 0.005*"apartment" + '
  '0.005*"stay"'),
 (6,
  '0.017*"room" + 0.008*"host" + 0.007*"

In [108]:
import os
import pickle
import pyLDAvis
import pyLDAvis.gensim_models

# Khai báo đường dẫn đến thư mục lưu trữ dữ liệu
LDAvis_data_dir = os.path.join("E:\\NCKH\\NCKHNew\\img", 'neg_Results')
if not os.path.exists(LDAvis_data_dir):
    os.makedirs(LDAvis_data_dir)

# Đường dẫn tới tệp dữ liệu của pyLDAvis
LDAvis_data_filepath = os.path.join(LDAvis_data_dir, 'ldavis_prepared_'+str(num_topics))

# Kiểm tra xem liệu quá trình chuẩn bị dữ liệu cho trực quan hóa đã được thực hiện chưa
if 1 == 1:
    # Chuẩn bị dữ liệu cho trực quan hóa
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    # Lưu trữ dữ liệu vào một tệp
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# Load dữ liệu đã chuẩn bị từ tệp
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

# Lưu trữ trực quan hóa dưới dạng một tệp HTML
pyLDAvis.save_html(LDAvis_prepared, 'ldavis_prepared_'+ str(num_topics) +'.html')

# Hiển thị trực quan hóa trên trình duyệt web
pyLDAvis.display(LDAvis_prepared)
